In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import numpy as np
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [25]:
# multiclassification task
# cnn


In [26]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [27]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [28]:
train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=12345,
                        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                        VALIDATION_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        class_mode='categorical')


Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [29]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [34]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)              
model.fit_generator(train_generator,
                    steps_per_epoch = num_steps,
                    epochs=10,
                    validation_data = validation_generator,
                    validation_steps = num_steps)

/var/folders/0y/fcwnfr4d06z44tl7ddmtdsrr0000gn/T/ipykernel_93405/670373825.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
8/8 [==============================] - 21s 3s/step - loss: 0.7909 - acc: 0.6680 - val_loss: 0.2186 - val_acc: 0.9280
Epoch 2/10
8/8 [==============================] - 18s 2s/step - loss: 0.2732 - acc: 0.8980 - val_loss: 0.1296 - val_acc: 0.9500
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: 0.1622 - acc: 0.9300 - val_loss: 0.1112 - val_acc: 0.9700
Epoch 4/10
8/8 [==============================] - 16s 2s/step - loss: 0.1280 - acc: 0.9580 - val_loss: 0.0944 - val_acc: 0.9600
Epoch 5/10
8/8 [==============================] - 17s 2s/step - loss: 0.0845 - acc: 0.9760 - val_loss: 0.0825 - val_acc: 0.9680
Epoch 6/10
8/8 [==============================] - 17s 2s/step - loss: 0.0886 - acc: 0.9720 - val_loss: 0.0778 - val_acc: 0.9640
Epoch 7/10
8/8 [==============================] - 17s 2s/step - loss: 0.0836 - acc: 0.9700 - val_loss: 0.0723 - val_acc: 0.9700
Epoch 8/10
8/8 [==============================] - 16s 2s/step - loss: 0.0631 - acc: 0.9840 - val_loss: 0

In [35]:
model.save('model.h5')

In [37]:
# VARIABLES
IMG_WIDTH, IMG_HEIGHT = 224, 224
VALIDATION_DATA_DIR = 'data/val'
VALIDATION_BATCH_SIZE = 64

# DATA GENERATORS
validation_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=VALIDATION_BATCH_SIZE,
        shuffle=False,
        class_mode='categorical')
ground_truth = validation_generator.classes

Found 500 images belonging to 2 classes.


In [44]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform

with CustomObjectScope(
    {'GlorotUniform': glorot_uniform()}):
    model = load_model('model.h5')

In [46]:
# Analysis

In [65]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [47]:
# Let's view the names of the files.
filenames = validation_generator.filenames
print(len(filenames))
print(filenames[:10])

500
['cat/cat.10060.jpg', 'cat/cat.10117.jpg', 'cat/cat.1024.jpg', 'cat/cat.10327.jpg', 'cat/cat.10346.jpg', 'cat/cat.10380.jpg', 'cat/cat.1041.jpg', 'cat/cat.10428.jpg', 'cat/cat.10434.jpg', 'cat/cat.10455.jpg']


In [48]:
ground_truth = validation_generator.classes
print(ground_truth[:10])
print(len(ground_truth))

[0 0 0 0 0 0 0 0 0 0]
500


In [49]:
label_to_index = validation_generator.class_indices
print(label_to_index)

{'cat': 0, 'dog': 1}


In [50]:
index_to_label = dict((i,j) for (j,i) in label_to_index.items())
print(index_to_label)

{0: 'cat', 1: 'dog'}


In [51]:
predictions = model.predict_generator(validation_generator,
                                      steps=None)

/var/folders/0y/fcwnfr4d06z44tl7ddmtdsrr0000gn/T/ipykernel_93405/4095933862.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(validation_generator,


In [52]:
print(predictions[:10])

[[9.9999356e-01 6.4100700e-06]
 [9.9999571e-01 4.2538677e-06]
 [9.9997973e-01 2.0304913e-05]
 [9.8737216e-01 1.2627833e-02]
 [9.9902391e-01 9.7609230e-04]
 [8.5441142e-01 1.4558858e-01]
 [9.9979740e-01 2.0260277e-04]
 [9.9998951e-01 1.0452185e-05]
 [9.9927860e-01 7.2139746e-04]
 [9.9992216e-01 7.7881436e-05]]


In [53]:
prediction_index = []
for prediction in predictions:
    prediction_index.append(np.argmax(prediction))

In [54]:
prediction_index

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [57]:
def accuracy(predictions, ground_truth):
    total = 0
    for i,j in zip(predictions, ground_truth):
        if i==j:
            total+=1
    return (total*1.0)/len(predictions)

In [58]:
accuracy(prediction_index, ground_truth)

0.974

In [59]:
# more compact analysis
prediction_table = {}
for index, val in enumerate(predictions):
    index_of_highest_probability = np.argmax(val)
    value_of_highest_probability = val[index_of_highest_probability]
    prediction_table[index] = [
        value_of_highest_probability, index_of_highest_probability,
        ground_truth[index]
    ]
assert len(predictions) == len(ground_truth) == len(prediction_table)

In [61]:
prediction_table # (probability, prediction, truth)

{0: [0.99999356, 0, 0],
 1: [0.9999957, 0, 0],
 2: [0.99997973, 0, 0],
 3: [0.98737216, 0, 0],
 4: [0.9990239, 0, 0],
 5: [0.8544114, 0, 0],
 6: [0.9997974, 0, 0],
 7: [0.9999895, 0, 0],
 8: [0.9992786, 0, 0],
 9: [0.99992216, 0, 0],
 10: [0.9946024, 0, 0],
 11: [0.9999589, 0, 0],
 12: [0.999967, 0, 0],
 13: [0.99969864, 0, 0],
 14: [0.99994016, 0, 0],
 15: [0.99999166, 0, 0],
 16: [0.99998367, 0, 0],
 17: [0.99998844, 0, 0],
 18: [0.999861, 0, 0],
 19: [0.98434246, 0, 0],
 20: [0.9994708, 0, 0],
 21: [0.9999635, 0, 0],
 22: [0.97714806, 1, 0],
 23: [0.99992025, 0, 0],
 24: [0.99876106, 0, 0],
 25: [0.94422436, 1, 0],
 26: [0.97457576, 0, 0],
 27: [0.9999409, 0, 0],
 28: [0.9999118, 0, 0],
 29: [0.9999949, 0, 0],
 30: [0.9999746, 0, 0],
 31: [0.9994424, 0, 0],
 32: [0.9999945, 0, 0],
 33: [0.99995375, 0, 0],
 34: [0.99995315, 0, 0],
 35: [0.9999976, 0, 0],
 36: [0.99994767, 0, 0],
 37: [0.99796236, 0, 0],
 38: [0.9968901, 0, 0],
 39: [0.9999746, 0, 0],
 40: [0.99969816, 0, 0],
 41: [0.